## Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of all 16 stamps for all 4 CCDs. \
Craig Lage - 15-Apr-25

In [ ]:
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCol = hdr0['ROICOL']
    roiRow = hdr0['ROIROW']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

# The below converts the amp name into the sequential numbering that comes
# from unpacking.
ampDict = {'C00':0, 'C01':1, 'C02':2, 'C03':3, 'C04':4,
           'C05':5, 'C06':6, 'C07':7, 'C10':8, 'C11':9,
           'C12':10, 'C13':11, 'C14':12, 'C15':13, 'C16':14,
           'C17':15}

# Build one frame for one CCD

In [ ]:
%matplotlib inline
camera = LsstCam.getCamera()

dayObs = 20250410
seqNum = 8
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R40'
ccd = 'SG1'
n = 2 # This just unpacks one frame.  This chooses which frame
detName = f"{raft}_{ccd}"
filename = f"s3://butler@rubinobs-raw-lsstcam/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
llX = roiCol
llY = roiRow
urX = roiCol + roiCols
urY = roiRow + roiRows

for detector in camera:
    if detector.getName()== detName:
        break
bbox = detector.getBBox()
nx,ny = bbox.getDimensions()
fullImage = np.zeros((ny,nx))
fullImage[:,:]=np.nan

hduNum = 2 * n + 1
hdrn = hdu_list[hduNum].header
timestamp = hdrn['STMPTIME']
image_out = unpackStamps(hduNum)
lct = LsstCameraTransforms(camera,detName)

for amp in detector.getAmplifiers():
    ampName = amp.getName()
    seg = ampDict[ampName]

    llCCDX,llCCDY = lct.ampPixelToCcdPixel(llX,llY,ampName)  # get CCD x,y these are floats
    llfpX,llfpY = lct.ampPixelToFocalMm(llX,llY,ampName)     # get focal plane x,y in mm

    urCCDX,urCCDY = lct.ampPixelToCcdPixel(urX,urY,ampName)
    urfpX,urfpY = lct.ampPixelToFocalMm(urX,urY,ampName)

    roiarr = image_out[seg]
    if urCCDX < llCCDX:
        roiarr = np.fliplr(roiarr)
    if urCCDY < llCCDY:
        roiarr = np.flipud(roiarr)
    
    # this doesn't workout how to map the stamps to the CCD, but the ll,ur points have enough info
    fullImage[int(min(llCCDY,urCCDY)):int(max(llCCDY,urCCDY)),
        int(min(llCCDX,urCCDX)):int(max(llCCDX,urCCDX))] = roiarr      


In [ ]:
x = plot(fullImage, stretch='ccs')